This doc run the data refresh process for the US48 demand for electricity dataset from the EIA API. It includes the following functionality:

- Check if new data available on the API
- Setting the data request parameters and pull the data
- Data quality checks 
- Appending the new data and saving the data
- Updating the log file 

## Load libraries

We will pull the data from the EIA API using a set of functions on the `eia_api.py` file. This includes the following functions:

- `eia_get` - A function for query data from the API. Can pull up to 5000 rows per call
- `eia_backfile` - A wrapper function, using batches requests from the API using the `eia_get` function to serve large requests (more than 5000 rows)
- `day_offset` - A helper function creates a vector of dates equally spaced by days
- `hour_offset` - A helper function creates a vector of dates equally spaced by days

The `eia_api.py` file imports the following libraries:

- `pandas` - for data processing
- `datetime` - to work with dates and time objects
- `requests` - to send `GET` requests to the EIA API


In addition, we will use the following libraries:

- `os` - load environment variables
- `numpy` - to create sequences (vectors)
- `plotly` - visualize the data
- `pathlib` - set file path
- `ydata_profiling` - for data profiling


In [ ]:
#| label: load libraries
import eia_api
import eia_etl as etl
import eia_forecast as fc_old
import dev_fc as fc
import os
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from statistics import mean
from pathlib import Path
from ydata_profiling import ProfileReport
from zoneinfo import ZoneInfo
from darts import TimeSeries
from darts.models.forecasting.linear_regression_model import LinearRegressionModel

## Load the Log File and API Metadata

We will load the log file from the `data` folder:


In [ ]:
log_file = etl.load_log(path = "data/us48_metadata.csv")

The `load_log` function returns the following objects:

- `log` - the full data log
- `last_success` - the log of the last success data pull
- `end` - the timestamp of the last data point derived from the `last_success` row 
- `start` - the start timestamp of the future data pull (calculated as `end` + 1 hour)


In [ ]:
print(log_file.last_success)
print(log_file.end)
print(log_file.start)

start = log_file.start

Setting the GET request parameters:


In [ ]:
api_path = "electricity/rto/region-data/data"
facets = {
    "respondent": "US48",
    "type": "D"
}

offset = 2250

eia_api_key = os.getenv('EIA_API_KEY')

Pulling the series metadata from the API. We will use it to check if new data is available on the API using the `get_api_end` function:


In [ ]:
metadata = etl.get_api_end(api_key = eia_api_key, 
api_path = "electricity/rto/region-data/", offset = 8)

The `end` variable represents the timestamp of the most recent data point available on the API. There is some issue on the API timestampe, a gap of 8 hours between the value of the last data point and the actual, therefore, we will offset the `end` value by 8 hours using the `offset` argument and save it as `end_fix`:


In [ ]:
print(metadata.end)
print(metadata.end_fix)

end_fix = metadata.end_fix

In [ ]:
new_data = etl.eia_data_refresh(start = start, end = end_fix, api_key = eia_api_key, api_path = api_path, facets = facets, offset = 24 * 7) 

## Updating the Dataset and Log File


In [ ]:
append_data = etl.append_new_data(data_path = "data/us48.csv",
                           log_path = "data/us48_metadata.csv",
                           new_data = new_data,
                           save = True)

data = append_data.data

In [ ]:
print(append_data.log)
print(append_data.data_update)

## Forecast Refresh

Forecast scope:
- Used a train forecasting model and refit it with new data
- Forecast refresh daily at the end of the day (past 23:00 UTC)
- Forecast horizon 24 hours

Steps:
- Load the forecast log file
- Check the time stamp of the most recent forecast
- Check if new data meet the refresh criteria
- If meet the criteria, update the forecast
- Update the log file



Forecast parameters


In [ ]:
md_params = fc.model_params(model = "LinearRegressionModel",
                    model_label = "model 1",
                    comments = "LM Model",
                    h = 24,
                    lags = [-24, -7 * 24, - 365 * 24],
                    freq = 24,
                    pi = 0.95,
                    train_length =  24 * 365 * 2,
                    num_samples= 500,
                    likelihood = "quantile",
                    quantiles = [0.025, 0.1, 0.25, 0.5, 0.75, 0.9, 0.975],
                    seed = 12345)  

mlflow_settings = fc.mlflow_params(path = "./metadata/",
                                experiment_name = "Forecast Dev",
                                type = "forecast",
                                score = False,
                                append = False,
                                version = "0.0.1")

In [ ]:
log_path = "data/fc48_metadata.csv"
fc_path = "data/fc48.csv"

forecast_start = data["period"].max().floor(freq = "d")
if os.path.isfile(log_path):
    last_start = fc.get_last_fc_start(log_path = log_path)
    create_new_fc = last_start < forecast_start
else:
    create_new_fc = True


if create_new_fc:
    print("Create new forecast")
    end = forecast_start - datetime.timedelta(hours = 1)
    start = end - datetime.timedelta(hours = md_params.train_length)
    print(start, end)
    input = fc.set_input(input = data, start = start, end = end)
    f = fc.forecast_object()
    f.add_input(input = input) 
    f.add_model_params(model_params =  md_params)    
    f.add_mlflow_settings(mlflow_settings = mlflow_settings)
    f.create_forecast()

    print(f.model_meta)
    log = fc.append_log(log_path= log_path, new_log = f.model_meta, save = True, init = True)
    new_fc = fc.append_forecast(fc_path =  fc_path, fc_new = f, save = True, init = True)

Score past forecasts


In [ ]:
forecast_archive = fc_old.load_forecast(fc_path)
new_log = fc_old.score_forecast(log_path = "data/fc48_metadata.csv", actual = data, forecast = forecast_archive, save = True)